In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import time

driver_path = r'C:\Users\rewha\4.Job Preparation_ACH 01_CODE\chromedriver-win64\chromedriver.exe'

service = Service(driver_path)
driver = webdriver.Chrome(service=service)

url = "https://en.wikipedia.org/wiki/Timeline_of_20th-century_history"
driver.get(url)

time.sleep(3)
html = driver.page_source
driver.quit()


In [2]:
soup = BeautifulSoup(html, "html.parser")
page_text = soup.get_text()

print(page_text[:1000])  # Small test print to verify it worked




Timeline of 20th-century history - Wikipedia


























Jump to content







Main menu





Main menu
move to sidebar
hide



		Navigation
	


Main pageContentsCurrent eventsRandom articleAbout WikipediaContact us





		Contribute
	


HelpLearn to editCommunity portalRecent changesUpload fileSpecial pages



















Search











Search






















Appearance





Appearance
move to sidebar
hide

TextSmallStandardLargeThis page always uses small font sizeWidthStandardWideThe content is as wide as possible for your browser window.Color (beta)AutomaticLightDarkThis page is always in light mode.












Donate

Create account

Log in








Personal tools





Donate Create account Log in





		Pages for logged out editors learn more



ContributionsTalk



























Timeline of 20th-century history



Add languages



						Page contents not supported in other languages.
					










ArticleTalk





English

















In [3]:
with open("20th_century_events.txt", "w", encoding="utf-8") as f:
    f.write(page_text)
